Training done on all homes from Austin, where we learn the home and the appliance factors. We reuse the same appliance factors in SD. We use the aggregate appliance factors to learn the home factors for SD, which we then use to predict the appliance energy.

In [1]:
%matplotlib inline
from common import APPLIANCES_ORDER, compute_rmse_fraction

In [2]:
from mf_core import *

import numpy as np
import pandas as pd
import sys
import os
from create_matrix import *
from sklearn.model_selection import LeaveOneOut
from sklearn.neighbors import KNeighborsRegressor
import pickle


from degree_days import dds
appliance_index = {appliance: APPLIANCES_ORDER.index(appliance) for appliance in APPLIANCES_ORDER}

APPLIANCES = ['fridge', 'hvac', 'wm', 'mw', 'oven', 'dw']

In [3]:
# Computing the best number of latent factors
import pickle
mf_all_p = pickle.load(open("predictions/o_mf_all_appliances.pkl",'r'))['mf_all_appliances']

mf_all_results = {}
feature='energy'

dataset = 'False'
cost = 'absolute'
for appliance in APPLIANCES_ORDER[1:]:
    print(appliance)
    mf_all_results[appliance] = {}
    
    for lat in [str(x) for x in range(1, 10)]:
        try:
            pred = mf_all_p[appliance][feature][cost][dataset][lat]
            if appliance=="hvac":
                pred = pred[['hvac_%d' %month for month in range(5, 11)]]
            alpha, beta, error, e  = compute_rmse_fraction(appliance, pred)
            mf_all_results[appliance][lat] = error
            
        except Exception, e:
            print(e)

hvac
'7'
'8'
'9'
fridge
'7'
'8'
'9'
mw
'7'
'8'
'9'
dw
'7'
'8'
'9'
wm
'7'
'8'
'9'
oven
'7'
'8'
'9'


In [4]:
num_lat = int(pd.DataFrame(mf_all_results).mean(axis=1).argmin())

In [5]:
import sys

features, cost, all_features, latent_factors = "energy", "absolute", False, num_lat
region = 'Austin'
year = 2014
df, dfc = create_matrix_single_region('Austin', 2014)
start, stop = 1, 13
energy_cols = np.array(
    [['%s_%d' % (appliance, month) for month in range(start, stop)] for appliance in APPLIANCES_ORDER]).flatten()

static_cols = ['area', 'total_occupants', 'num_rooms']
static_df = df[static_cols]
static_df = static_df.div(static_df.max())
weather_values = np.array(dds[2014][region][start - 1:stop - 1]).reshape(-1, 1)

dfc = df.copy()

df = dfc[energy_cols]
col_max = df.max().max()
col_min = df.min().min()

X_matrix, X_normalised, matrix_max, matrix_min = preprocess_all_appliances(df, dfc)
static_features = get_static_features(dfc, X_normalised)
feature_comb = ['None']
idx_user, data_user = prepare_known_features(feature_comb, static_features, X_normalised)

test_ix = 0

test_home = df.index.values[test_ix]
A = create_matrix_factorised_all_appliances([test_home], X_normalised)
X, Y, res = nmf_features(A=A, k=latent_factors, constant=0.01, regularisation=False,
                         idx_user=idx_user, data_user=data_user,
                         idx_item=None, data_item=None, MAX_ITERS=15, cost=cost)

In [6]:
pred = pd.DataFrame(Y*X)
pred.index = df.index
pred.columns = X_normalised.columns

In [7]:
austin_appliance_factors = np.array(X)
austin_aggregate_factors = austin_appliance_factors[:, :12]

In [8]:
sd_df, sd_dfc = create_matrix_single_region('SanDiego', 2014)
start, stop = 1, 13
energy_cols = np.array(
    [['%s_%d' % (appliance, month) for month in range(start, stop)] for appliance in APPLIANCES_ORDER]).flatten()

static_cols = ['area', 'total_occupants', 'num_rooms']
static_df = sd_df[static_cols]
static_df = static_df.div(static_df.max())
weather_values = np.array(dds[2014][region][start - 1:stop - 1]).reshape(-1, 1)

sd_dfc = sd_df.copy()

sd_df = sd_dfc[energy_cols]

In [9]:
agg_sd_energy = sd_df.values[:,:12]
from scipy.optimize import nnls

In [10]:
m, n  = agg_sd_energy.shape
k, _ = austin_aggregate_factors.shape
sd_homes_factors = np.zeros((m ,k))
for j in range(m):
    mask_rows = pd.Series(agg_sd_energy[j,:]).notnull()
    sd_homes_factors[j,:] = nnls(austin_aggregate_factors.transpose()[mask_rows], agg_sd_energy[j,:][mask_rows])[0]

In [12]:
pred_df_sd = pd.DataFrame(np.dot(sd_homes_factors, austin_appliance_factors))

In [13]:
pred_df_sd.columns = sd_df.columns
pred_df_sd.index = sd_df.index

In [14]:
from common import compute_rmse_fraction

In [15]:
mf_res = {}
for appliance in APPLIANCES_ORDER[1:]:
    if appliance=="hvac":
        start, stop = 5, 11
    else:
        start, stop = 1, 13
    mf_res[appliance] = compute_rmse_fraction(appliance, pred_df_sd[['%s_%d' %(appliance, month) for month in range(start, stop)]], 'SanDiego')[2]

In [16]:
pd.Series(mf_res)

dw        97.077255
fridge    10.572350
hvac      26.184838
mw        62.521991
oven       4.527526
wm        36.853539
dtype: float64